**Customer lifetime value (CLV or CLTV)** is a metric that represents the total net profit a company can expect to generate from a customer throughout their entire relationship.

The goal of this project is to model Customer Lifetime Value(CLV) using 2 approaches: Cohort Model (Historical Approach) and Machine Learning Model (Predictive Approach).

CLV is calculated using the below formula :
           CLV = ((Average Sales x Purchase Frequency)/Churn Rate) x Profit margin
           where,
               Average Sales = Total Sales/Total No of Orders,
               Purchase Frequency = Total No of Orders/Total Unique Customers
               Retention Rate = Total No of Orders>1/Total Unique Customers
               Churn Rate = 1-Retention Rate
               Profit Margin -> Based on business context. Here I am taking it as 6%

## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Loading Dataset

In [2]:
df = pd.read_excel('C://Users//durga//Analytics Projects//Dataset//OnlineRetail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Calculating Total Sales

In [4]:
# Calculation the total sales
df['TotalSales'] = df['Quantity']*df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### Data Exploration and Cleaning 

In [5]:
df.shape

(541909, 9)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   TotalSales   541909 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 37.2+ MB


In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
TotalSales          0
dtype: int64

In [8]:
df.describe()

,Quantity,UnitPrice,CustomerID,TotalSales
count,541909.000000,541909.000000,406829.000000,541909.000000
mean,9.552250,4.611114,15287.690570,17.987795
std,218.081158,96.759853,1713.600303,378.810824
min,-80995.000000,-11062.060000,12346.000000,-168469.600000
25%,1.000000,1.250000,13953.000000,3.400000
50%,3.000000,2.080000,15152.000000,9.750000
75%,10.000000,4.130000,16791.000000,17.400000
max,80995.000000,38970.000000,18287.000000,168469.600000


In [9]:
# Removing entries with Quantity < 0
df=df[df['Quantity']>0]

# Filtering entries with UnitPrice > 0
df=df[df['UnitPrice']>0]
df.describe()

,Quantity,UnitPrice,CustomerID,TotalSales
count,530104.000000,530104.000000,397884.000000,530104.000000
mean,10.542037,3.907625,15294.423453,20.121871
std,155.524124,35.915681,1713.141560,270.356743
min,1.000000,0.001000,12346.000000,0.001000
25%,1.000000,1.250000,13969.000000,3.750000
50%,3.000000,2.080000,15159.000000,9.900000
75%,10.000000,4.130000,16795.000000,17.700000
max,80995.000000,13541.330000,18287.000000,168469.600000


In [10]:
# Removing CustomerID records with null values
df = df.dropna(subset=['CustomerID'])
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397884 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
 8   TotalSales   397884 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB


In [13]:
# Details of the transactions
mindate = df['InvoiceDate'].dt.date.min()
maxdate = df['InvoiceDate'].dt.date.max()
unique_cust = df['CustomerID'].nunique()
qty_sold = df['Quantity'].sum()
tot_sales = df['TotalSales'].sum()

print(f"Range of transactions: {mindate} to {maxdate}")
print(f"No of unique customers: {unique_cust}")
print(f"Total Quantity sold: {qty_sold}")
print(f"Total Sales for the period: {tot_sales}")

Range of transactions: 2010-12-01 to 2011-12-09
No of unique customers: 4338
Total Quantity sold: 5167812
Total Sales for the period: 8911407.904


## Cohort Model

In Cohort Model, the customers are split into several groups called as cohorts instead of taken as individuals. Each person in a cohort must share a related yet distinguishable trait that separates them from the other cohorts.The most common way to group customers into cohorts is by the start date of a customer, typically by month.
For this particular project, the customers are grouped based on their start month which means 12 cohorts of the customers (Jan-Dec).

In [14]:
# Customers are grouped for analysis
customers = df.groupby('CustomerID').agg({'InvoiceDate':lambda x: x.min().month,
                                          'InvoiceNo': lambda x:len(x),
                                          'TotalSales': lambda x:np.sum(x)})

customers.columns = ['StartMonth','Frequency', 'TotalSales']
customers.head()

,StartMonth,Frequency,TotalSales
CustomerID,,,
12346.0,1,1,77183.60
12347.0,12,182,4310.00
12348.0,12,31,1797.24
12349.0,11,73,1757.55
12350.0,2,17,334.40


In [15]:
# Calculating CLV for each cohort
monthly_CLV = []
profit_margin = 0.06

for i in range(1,13):
    cust_m = customers[customers['StartMonth']==i]
    avg_sales = round(np.mean(cust_m['TotalSales']),2)
    purch_freq = round(np.mean(cust_m['Frequency']),2)
    retention_rate = cust_m[cust_m['Frequency']>1].shape[0]/cust_m.shape[0]
    churn = round(1-retention_rate,2)
    
    CLV = round((avg_sales*purch_freq/churn)*profit_margin,2)
    
    monthly_CLV.append(CLV)   

In [16]:
# Creating dataframe to store CLV for each cohort
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

data_clv = pd.DataFrame({'Month': months,
                        'CLV': monthly_CLV})
data_clv

,Month,CLV
0,Jan,1855844.51
1,Feb,375688.70
2,Mar,647919.81
3,Apr,419056.03
4,May,177116.99
5,Jun,388554.92
6,Jul,86112.51
7,Aug,178601.30
8,Sep,222186.84
9,Oct,172851.54


There are 12 CLV values for the 12 months and the new customers will have different CLV values attached to them based on the moonth they join. Each group will behave differently and thus different campaigns can be used for each customers.

## Machine Learning Model

This includes a regression model for the existing customers, taking data for 8 months as independent variables.

In [17]:
# Filtering columns to create a dataframe for ML model
ml_data = df[['CustomerID','InvoiceDate','InvoiceNo','Quantity','TotalSales']]
ml_data['MonthYear'] = ml_data['InvoiceDate'].apply(lambda x:x.strftime('%b-%Y'))
ml_data.head()

C:\Users\durga\AppData\Local\Temp\ipykernel_20104\2185930935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_data['MonthYear'] = ml_data['InvoiceDate'].apply(lambda x:x.strftime('%b-%Y'))


,CustomerID,InvoiceDate,InvoiceNo,Quantity,TotalSales,MonthYear
0,17850.0,2010-12-01 08:26:00,536365,6,15.30,Dec-2010
1,17850.0,2010-12-01 08:26:00,536365,6,20.34,Dec-2010
2,17850.0,2010-12-01 08:26:00,536365,8,22.00,Dec-2010
3,17850.0,2010-12-01 08:26:00,536365,6,20.34,Dec-2010
4,17850.0,2010-12-01 08:26:00,536365,6,20.34,Dec-2010


In [25]:
# Creating a pivot table to take the months as columns and sales value as data
ml_pivot = ml_data.pivot_table(values='TotalSales', index=['CustomerID'],columns=['MonthYear'],aggfunc='sum',fill_value=0).reset_index()
ml_pivot.head()

MonthYear,CustomerID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011
0,12346.0,0.00,0.00,0.00,0.00,0.0,77183.60,0.0,0.00,0.0,0.0,0.00,0.00,0.0
1,12347.0,636.25,584.91,711.79,224.82,0.0,475.39,0.0,382.52,0.0,0.0,0.00,1294.32,0.0
2,12348.0,367.00,0.00,892.80,0.00,0.0,227.44,0.0,0.00,0.0,0.0,0.00,0.00,310.0
3,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1757.55,0.00,0.0
4,12350.0,0.00,0.00,0.00,0.00,334.4,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0


In [26]:
# Calculating sum of totalsales of all months
ml_pivot['CLV'] = ml_pivot.iloc[:,1:].sum(axis=1)
ml_pivot.head()

MonthYear,CustomerID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011,CLV
0,12346.0,0.00,0.00,0.00,0.00,0.0,77183.60,0.0,0.00,0.0,0.0,0.00,0.00,0.0,77183.60
1,12347.0,636.25,584.91,711.79,224.82,0.0,475.39,0.0,382.52,0.0,0.0,0.00,1294.32,0.0,4310.00
2,12348.0,367.00,0.00,892.80,0.00,0.0,227.44,0.0,0.00,0.0,0.0,0.00,0.00,310.0,1797.24
3,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1757.55,0.00,0.0,1757.55
4,12350.0,0.00,0.00,0.00,0.00,334.4,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,334.40


In [36]:
# Defining features (X) and target variable (y) - considering data for 8 months
X = ml_pivot[['May-2011','Jun-2011','Jul-2011','Aug-2011','Sep-2011','Oct-2011','Nov-2011','Dec-2011']]
y = ml_pivot[['CLV']]

In [37]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [38]:
# Training Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [39]:
# Making Predictions on test set
y_pred = model.predict(X_test)

In [40]:
# calculate r-squared for the model
print("R-squared:",r2_score(y_test,y_pred))

# calculate Mean Absolute Error
print("MAE:",mean_absolute_error(y_test,y_pred))

#calculate Mean Squared Error
print("MSE",mean_squared_error(y_test, y_pred))

# compute the RMSE of the predictions
print("RMSE:",np.sqrt(mean_squared_error(y_test, y_pred)))

R-squared: 0.9818496414642452
MAE: 517.5090333034678
MSE 1859374.128389391
RMSE: 1363.5886947277727


The r-squared value is high which shows the model fits the data very well. However, there are many ways to find CLV and effective utilization of the value to create strategies can be effective to scale the business.